In [8]:
import json

In [4]:
ssc_old_id_to_new_id = dict()
with open('../data/ssc.id_map.from.repository', 'r') as f:
    for line in f:
        pieces = line.strip().split('\t')
        ssc_old_id_to_new_id[pieces[1]] = pieces[0]
        ssc_old_id_to_new_id[pieces[1].replace('.', '_')] = pieces[0]

In [5]:
ped_file = '../../DATA/ssc/ssc.ped'
# pull affected status
# (0=unknown; 1=unaffected; 2=affected)
child_id_to_affected = dict()
child_id_to_sex = dict()
fam_to_inds = dict()
with open(ped_file, 'r') as f:
    for line in f:
        pieces = line.strip().split('\t')
        if len(pieces) >= 6:
            fam_id, child_id, f_id, m_id, sex, disease_status = pieces[0:6]
            if child_id in ssc_old_id_to_new_id and m_id in ssc_old_id_to_new_id and f_id in ssc_old_id_to_new_id:
                child_id = ssc_old_id_to_new_id[child_id]
                f_id = ssc_old_id_to_new_id[f_id]
                m_id = ssc_old_id_to_new_id[m_id]
                child_id_to_affected[child_id] = disease_status
                child_id_to_sex[child_id] = sex

                if (fam_id, m_id, f_id) not in fam_to_inds:
                    fam_to_inds[(fam_id, m_id, f_id)] = [m_id, f_id]
                fam_to_inds[(fam_id, m_id, f_id)].append(child_id)
                
print('ped loaded')
print('children', len(child_id_to_affected))

ped loaded
children 1037


In [7]:
sebat_deletions = []
child_to_parents = dict()
for family, inds in fam_to_inds.items():
    for child in inds[2:]:
        child_to_parents[child] = (family[0], tuple(inds[:2]))
        
trio_to_ismatpat = {
    ('0/0', '0/0', '0/1'): (False, False),
    ('0/0', '0/0', '1/1'): ((False, False), (False, False)),
    ('0/0', '0/1', '0/1'): (False, True),
    ('0/0', '0/1', '1/1'): ((False, True), (False, False)),
    ('0/0', '1/1', '0/1'): (False, True),
    ('0/0', '1/1', '1/1'): ((False, True), (False, False)),
    ('0/0', './.', '0/1'): (False, False),
    ('0/0', './.', '1/1'): ((False, False), (False, False)),
    ('0/1', '0/0', '0/1'): (True, False),
    ('0/1', '0/0', '1/1'): ((True, False), (False, False)),
    ('0/1', '0/1', '0/1'): (True, True),
    ('0/1', '0/1', '1/1'): ((True, False), (False, True)),
    ('0/1', '1/1', '0/1'): (False, True),
    ('0/1', '1/1', '1/1'): ((True, False), (False, True)),
    ('0/1', './.', '0/1'): (True, False),
    ('0/1', './.', '1/1'): ((True, False), (False, False)),
    ('1/1', '0/0', '0/1'): (True, False),
    ('1/1', '0/0', '1/1'): ((True, False), (False, False)),
    ('1/1', '0/1', '0/1'): (True, False),
    ('1/1', '0/1', '1/1'): ((True, False), (False, True)),
    ('1/1', '1/1', '0/1'): (True, True),
    ('1/1', '1/1', '1/1'): ((True, False), (False, True)),
    ('1/1', './.', '0/1'): (True, False),
    ('1/1', './.', '1/1'): ((True, False), (False, False)),
    ('./.', '0/0', '0/1'): (False, False),
    ('./.', '0/0', '1/1'): ((False, False), (False, False)),
    ('./.', '0/1', '0/1'): (False, True),
    ('./.', '0/1', '1/1'): ((False, True), (False, False)),
    ('./.', '1/1', '0/1'): (False, True),
    ('./.', '1/1', '1/1'): ((False, True), (False, False)),
    ('./.', './.', '0/1'): (False, False),
    ('./.', './.', '1/1'): ((False, False), (False, False)),

}
sebat_families = []
with open('../data/Sebat_2017_SV_Callet.vcf', 'r') as f:
    # skip header
    header = next(f)
    while header.startswith('##'):
        #print(header)
        header = next(f)
    assert header.startswith('#CHROM')
    header = header.strip().split()
    samples = [x.split('|')[1] for x in header[9:]]
    sample_to_index = dict([(x, i) for i, x in enumerate(samples)])
    
    for family_id, inds in fam_to_inds.items():
        indices = [sample_to_index[x] for x in inds if x in sample_to_index]
        if len(indices)==4:
            sebat_families.append(family_id[0])
    print('families', len(sebat_families))

    for line in f:
        pieces = line.strip().split('\t', maxsplit=5)
        if pieces[4]=='<DEL>':
            pieces = line.strip().split('\t')
            
            info = [x for x in pieces[7].split(';')]
            start_pos = int(pieces[1])
            end_pos = int([x for x in info if x.startswith('END=')][0][4:])
            if 'CIPOS' in info:
                ci_pos = [int(y) for y in [x for x in info if x.startswith('CIPOS=')][0][6:].split(',')]
            else:
                ci_pos = (0, 0)
            if 'CIEND' in info:
                ci_end = [int(y) for y in [x for x in info if x.startswith('CIEND=')][0][6:].split(',')]
            else:
                ci_end = (0, 0)
            del_length = end_pos - start_pos
            
            for ind, gen in zip(samples, pieces[9:]):
                if (gen == '0/1' or gen == '1/1') and ind in child_to_parents:
                    family, (mom, dad) = child_to_parents[ind]
                    mom_gen, dad_gen = pieces[9:][sample_to_index[mom]], pieces[9:][sample_to_index[dad]] 
                    if gen == '0/1' and (mom_gen, dad_gen, gen) in trio_to_ismatpat:
                        is_mat, is_pat = trio_to_ismatpat[(mom_gen, dad_gen, gen)]
                        sebat_deletions.append({
                                'family': family,
                                'trans': [ind],
                                'chrom': pieces[0],
                                'start_pos': start_pos,
                                'end_pos': end_pos,
                                'length': del_length,
                                'opt_start_pos': start_pos+ci_pos[0],
                                'opt_end_pos': end_pos+ci_end[1],
                                'is_mat': is_mat,
                                'is_pat': is_pat,
                                'mother': mom,
                                'father': dad,
                                'is_inherited': is_mat or is_pat,
                                'is_denovo': not is_mat and not is_pat
                        })
                    elif gen == '1/1' and (mom_gen, dad_gen, gen) in trio_to_ismatpat:
                        for is_mat, is_pat in trio_to_ismatpat[(mom_gen, dad_gen, gen)]:
                            sebat_deletions.append({
                                    'family': family,
                                    'trans': [ind],
                                    'chrom': pieces[0],
                                    'start_pos': start_pos,
                                    'end_pos': end_pos,
                                    'length': del_length,
                                    'opt_start_pos': start_pos+ci_pos[0],
                                    'opt_end_pos': end_pos+ci_end[1],
                                    'is_mat': is_mat,
                                    'is_pat': is_pat,
                                    'mother': mom,
                                    'father': dad,
                                    'is_inherited': is_mat or is_pat,
                                    'is_denovo': not is_mat and not is_pat
                            })
        
print(len(sebat_deletions))

families 517
2340252


In [10]:
sebat_deletions = [d for d in sebat_deletions if d['length']>=100]
print('remaining deletions', len(sebat_deletions))

remaining deletions 1440092


In [12]:
sebat_deletions = [d for d in sebat_deletions if d['is_inherited']]
print('remaining deletions', len(sebat_deletions))

remaining deletions 1420888


In [13]:
with open('../phased_ssc.sebat_del/deletions.json', 'w+') as f:
    json.dump(sebat_deletions, f, indent=4)

In [15]:
with open('../phased_ssc.sebat_del/families.json', 'w+') as f:
    json.dump(sorted(sebat_families), f, indent=4)

In [18]:
with open('../phased_ssc.sebat_del/info.json', 'w+') as f:
    json.dump({"ped_file": "../DATA/ssc/ssc.ped", 
               "assembly": '37',
               "data_dir": None, 
               "param_file": None, 
               "num_loss_regions": None, 
               "detect_inherited_deletions": True, 
               "detect_denovo_deletions": False, 
               "chrom": None, 
               "detect_consanguinity": False}, f, indent=4)